In [29]:
import os
import json
import pprint
import requests
import traceback
from tqdm import tqdm
import numpy as np
import numpy.random as npr
import spacy
from bs4 import BeautifulSoup
import dbpedia_utils as dbu

In [6]:
path = 'data/grouped_chats.json'

with open(path, 'r') as file:
    data = json.load(file)
    i = 0
    for key, value in data.items():
        print(len(value))
        pprint.pprint(value)
        i += 1
        if i == 5:
            break

24
[{'fromBot': False,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_request',
  'request': {'messageData': [{'payload': 'start'}],
              'messageType': 'parameter',
              'platform': 'Web',
              'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
  'response': nan,
  'timestamp': 1553869389296.0,
  'userId': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3'},
 {'fromBot': True,
  'id': 'dbfaba55-ecc0-42a3-b2c7-1daa39fb36b3_1553869389296_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'I am the DBpedia Bot'}],
                'messageType': 'text'},
               {'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',


                                             'value': 'Berlin'},
                                            {'isShort': True,
                                             'name': 'City',
                                             'value': 'Berlin'},
                                            {'isShort': True,
                                             'name': 'Latitude',
                                             'value': '52.5170365'},
                                            {'isShort': True,
                                             'name': 'Longitude',
                                             'value': '13.3888599'}],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': 'Berlin, 10117, Deutschland',
                                 'title': 'Berlin (Location Info)'}],
                'messageType': 'generic'},
               {'messageData': [{'button

               {'messageData': [{'buttons': [{'buttonType': 'link',
                                              'slackStyle': 'default',
                                              'title': 'View Map',
                                              'uri': 'http://nominatim.openstreetmap.org/search?q=Hawaii&addressdetails=1&extratags=1&limit=1'},
                                             {'buttonType': 'param',
                                              'slackStyle': 'default',
                                              'title': 'Basic Information',
                                              'uri': 'entity-information__http://dbpedia.org/resource/Hawaii'}],
                                 'fields': [{'isShort': True,
                                             'name': 'Country',
                                             'value': 'United States of '
                                                      'America'},
                                            {'isShort'

                'messageType': 'smart_reply'}],
  'timestamp': 1553665542592.0,
  'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
 {'fromBot': False,
  'id': '03b12284-8cc5-47e2-9786-2319ee6b32ea_1553665550686_request',
  'request': {'messageData': [{'text': 'alright'}],
              'messageType': 'text',
              'platform': 'Web',
              'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
  'response': nan,
  'timestamp': 1553665550686.0,
  'userId': '03b12284-8cc5-47e2-9786-2319ee6b32ea'},
 {'fromBot': True,
  'id': '03b12284-8cc5-47e2-9786-2319ee6b32ea_1553665550686_response',
  'request': nan,
  'response': [{'messageData': [{'buttons': [],
                                 'fields': [],
                                 'image': 'http://i.imgur.com/044oUKK.png',
                                 'smartReplies': [],
                                 'text': "I'm not sure I understand you "
                                         'fully.'}],
                'messageTyp

                                             'values': {'https://en.wikipedia.org/wiki/German_Empire': 'German '
                                                                                                       'Empire',
                                                        'https://en.wikipedia.org/wiki/Kingdom_of_Württemberg': 'Kingdom '
                                                                                                                'of '
                                                                                                                'Württemberg',
                                                        'https://en.wikipedia.org/wiki/Ulm': 'Ulm'}},
                                            {'isShort': True,
                                             'name': 'Death Date',
                                             'value': '1955-04-18'},
                                            {'isShort': True,
                                            

In [7]:
# most used platform 
from collections import Counter

platform_count = Counter()
err = 0

for key, val in data.items():
    msg = val[0]
    if type(msg['request']) == float: # check if msg.request = nan
        err += 1
#         pprint.pprint(msg)
    else:
        platform_count[msg['request']['platform']] += 1
            
print(platform_count)
print(err)

Counter({'Web': 9078, 'Slack': 4, 'Facebook Messenger': 1})
1


In [8]:
# no. of failed answers per conversation and questions that bot couldn't answer
from collections import defaultdict

failed_qid_per_user = defaultdict(set)
failed_qid = set()
num_requests = 0 # includes START payload from user  
passed = 0
feedback_asked = 0 

for uid, conv in data.items():
    for msg in conv: # msg object
        if not msg['fromBot']: 
            num_requests += 1
            for obj in msg['request']['messageData']:
                if 'payload' in obj and 'feedback__no__' in obj['payload']:
                    failed_qid_per_user[uid].add(obj['payload'].replace('feedback__no__', ''))
                    failed_qid.add(obj['payload'].replace('feedback__no__', ''))
                    break
                elif 'payload' in obj and 'feedback__yes__' in obj['payload']:
                    passed += 1
                    break
                
        else:
            if type(msg['response']) != float:
                for obj in msg['response']:
                    if obj['messageType'] == 'smart_reply':
                        feedback_asked +=1
                        break


print('num_requests: ', num_requests)
print('num_failed: ', len(failed_qid))
print('convos with failed responses: ', len(failed_qid_per_user))
print('num_passed: ', passed)
print('num_asked: ', feedback_asked)

num_requests:  45797
num_failed:  4155
convos with failed responses:  1982
num_passed:  3406
num_asked:  28953


In [9]:
# questions that the bot couldn't answer
unanswered_ques = []

for uid, conv in data.items():
    if uid in failed_qid_per_user:
        for msg in conv: # msg object
            if not msg['fromBot']:
                if msg['id'].replace('_request', '') in failed_qid_per_user[uid]:
                    for obj in msg['request']['messageData']:
                        print(obj)
                        if 'text' in obj:
#                             print(obj)
                            unanswered_ques.append(obj['text'])
                            #break

print(len(unanswered_ques))

{'text': 'Who is George MacDonald Church ?'}
{'text': 'Who is Macron ?'}
{'payload': 'dbpedia-about'}
{'text': 'What is the plot of Titanic?'}
{'payload': 'dbpedia-about'}
{'text': 'who is sachin>'}
{'text': 'what is anthem of india?'}
{'text': 'what is life'}
{'text': 'How many bones does a cat have?'}
{'text': 'How to care a newborn?'}
{'text': 'How to care for newborn?'}
{'text': 'Where was Angela Merkel born '}
{'text': 'who is Angela Merkel?'}
{'text': 'Tell me something about Rachmaninov'}
{'text': 'what is konami'}
{'text': 'hello'}
{'text': 'where was george clooney born?'}
{'text': 'who is the mayor of Varese?'}
{'text': 'What is the German Empire?'}
{'text': 'who is einstein'}
{'text': 'Programming Language'}
{'text': 'how many members does the Beatles have?'}
{'text': 'The top biggest cities'}
{'text': 'The most difficult problem '}
{'text': 'heloo'}
{'payload': 'help'}
{'payload': 'dbpedia-about'}
{'payload': 'help'}
{'payload': 'dbpedia-contribute'}
{'payload': 'entity-inf

{'text': 'where are you'}
{'text': 'What is Paradox Interactive'}
{'text': 'Where is York'}
{'text': 'what is a planet?'}
{'text': 'What is a car'}
{'text': 'how does a fridge work'}
{'text': 'what did einstein discover'}
{'text': 'information extraction'}
{'text': 'what is the weather like tomorrow'}
{'text': 'What is on August 15 in India'}
{'text': 'August 15'}
{'text': 'create an appointment'}
{'text': 'Name the Prime Minister of India'}
{'text': 'what is sansara wheel?'}
{'text': 'what is sansara?'}
{'text': 'What is a computer?'}
{'text': 'what is Xuân Hương?'}
{'text': 'Where is Xuân Hương?'}
{'text': 'who is the leader of denmark'}
{'text': 'who is pia kjærsgård'}
{'text': 'Lars Løkke Rasmussen'}
{'text': 'what is the cathedral of grosseto?'}
{'text': 'Who is charles correa?'}
{'text': 'internet'}
{'text': 'is Alessandro Del Piero dead?'}
{'text': 'Is Kurt Cobain dead?'}
{'text': 'Are there images of Einstein?'}
{'text': 'What are the elevations of all the Azores islands'}
{'te

In [10]:
# get convos as request-response pairs, irrespective of userId  
utter = dict() # { qid : {req : [response]}}

for uid, conv in data.items():
    for msg in conv: # msg object
        if type(msg['id']) != float and '_request' in msg['id']: # request-from-user, add to dict
            qid = msg['id'].replace('_request', '')
            req_ob = msg['request']['messageData'][0]
            for key in req_ob:
                request = req_ob[key]
                obj = {request : []}
                utter[qid] = obj # every req-response pair is marked by a unique id 
                
        elif type(msg['id']) != float:
            resp_id = msg['id'].replace('_response', '')
            resp_list = msg['response']
            response = []
            if resp_id in utter:
                for obj in resp_list:
                    for text_ob in obj['messageData']:
                        if 'text' not in text_ob:
                            response.append(text_ob['title'])
                        else:
                            response.append(text_ob['text'])
                       
                for key in utter[resp_id]:
                    utter[resp_id][key].extend(response)
                    
with open('data/utterances.json', 'w') as outfile:
    json.dump(utter, outfile)


In [11]:
print(len(utter))

45797


In [12]:
# input data for clustering

import spacy 
nlp = spacy.load("en_core_web_sm")

req_pos = {}
for id in utter:
    for req in utter[id]:
        if req != 'start' and 'feedback' not in req:
#             print(req)
            doc = nlp(req)
#             req_pos[req] = [(chunk.text, chunk.root.head.text) for chunk in doc.noun_chunks] # with noun chunks
#             print([(chunk.text, chunk.root.head.text) for chunk in doc.noun_chunks])
            for token in doc:
                if token.dep_ == 'ROOT': # removable 
                    req_pos[req] = [(child, token) for child in token.children if not child.is_punct] # w/o noun chunks
                    print(req_pos[req])
    

[(Who, is), (Church, is)]
[(Who, is), (Macron, is)]
[(Who, is), (Macron, is)]
[(Who, is), (Einstein, is)]
[(Albert_Einstein, entity)]
[(WHER, IS), (PARIS, IS)]
[(WHERE, PARIS), (IS, PARIS)]
[(WHO, IS), (CHURCH, IS)]
[(WHO, IS), (MACRON, IS)]
[(Who, is), (Ming'swife, is)]
[(Albert_Einstein, entity)]
[(Planet, entity)]
[(get, location__Berlin)]
[(about, dbpedia)]
[(Albert_Einstein, entity)]
[(more__http://dbpedia.org, learn), (Einstein, learn)]
[(similar__http://dbpedia.org, load), (Einstein, load)]
[(get, location__Berlin)]
[(Planet, entity)]
[(Berlin, entity)]
[(more__http://dbpedia.org, learn), (resource, learn)]
[(similar__http://dbpedia.org, load), (resource, load)]
[(related__http://dbpedia.org, load), (resource, load)]
[(about, dbpedia)]
[(Albert_Einstein, entity)]
[(get, location__Berlin)]
[(more__http://dbpedia.org, learn), (Einstein, learn)]
[(who, is), (obama, is)]
[(get, location__Berlin)]
[(where, is), (newyork, is)]
[(Where, is), (Beijing, is)]
[(Where, is), (Hawaii, is)]
[

[(in, towns)]
[(in, towns)]
[(in, cities)]
[(Abbeville, County)]
[(Albert_Einstein, entity)]
[(get, location__Berlin)]
[(country, has), (population, has)]
[(where, is), (bengalure, is)]
[(which, is), (country, is)]
[(what, is), (population, is)]
[(about, dbpedia)]
[(which, is), (city, is)]
[(bigger, is), (kolkata, is)]
[(who, is), (chairman, is)]
[(Planet, entity)]
[(what, is), (dwarf, is)]
[(dwarf, is)]
[(large, is), (dwarf, is)]
[(what, is), (vertloop, is)]
[(what, is), (object, is)]
[(green, cloud), (space, cloud)]
[(green, object), (in, object)]
[(cat, is), (mammla, is)]
[(cat, is), (mammal, is)]
[]
[(of, density)]
[(what, is), (density, is)]
[(who, discovered), (pluto, discovered)]
[(who, was), (scientist, was)]
[]
[(Who, are), (you, are)]
[(What, ask), (can, ask), (I, ask), (you, ask)]
[(Albert_Einstein, entity)]
[(How, work), (do, work), (you, work)]
[(What, are), (you, are)]
[(How, work), (do, work), (you, work)]
[(children, had), (einstein, had)]
[(What, is), (building, is)]
[

[(language, spoken), (is, spoken), (in, spoken)]
[(who, are), (you, are)]
[(what, is), (planet, is)]
[(who, is), (minister, is)]
[(who, is), (captain, is)]
[(which, curry), (steph, curry), (plays, curry), (for, curry)]
[(which, curry), (steph, curry), (plays, curry), (for, curry)]
[(where, is), (india, is)]
[(where, is), (view, is)]
[(what, is), (capital, is)]
[(who, scored), (touchdowns, scored), (ever, scored)]
[(Planet, entity)]
[(Who, is), (Char, is)]
[(Who, is), (Char, is)]
[(Who, is), (Trump, is)]
[(What, think), (do, think), (you, think), (about, think)]
[(Who, is), (Chirac, is)]
[(WHO, IS), (CHIRAC, IS)]
[(about, dbpedia)]
[(who, is), (queen, is)]
[(what, is), (single, is)]
[(what, is), (movie, is)]
[(who, are), (council, are)]
[(films, been), (has, been), (in, been)]
[(what, is), (capital, is)]
[(how, find), (do, find), (you, find), (area, find)]
[(salty, is), (sea, is)]
[(states, is), (democracy, is)]
[(of, population)]
[(of, population)]
[(who, won), (awards, won)]
[(who, wo

[(Who, is), (Einstein, is)]
[(Who, is), (Khan, is)]
[(Who, is), (Hitler, is)]
[(Where, is), (Hitler, is)]
[(me, give), (information, give)]
[(Who, is), (Pakistan, is)]
[(Where, is), (Pakistan, is)]
[(What, is), (Google, is)]
[(What, is), (Earth, is)]
[(Where, is), (Karachi, is)]
[(WHO, IS), (EINSTEIN, IS)]
[(who, is), (einstein, is)]
[(Who, is), (Einstein, is)]
[(Albert_Einstein, entity)]
[(Who, is), (Einstein, is)]
[(Who, is), (Einstein, is)]
[(Where, is), (Dame, is)]
[(Where, is), (Statue, is)]
[]
[(Albert_Einstein, entity)]
[(Who, is), (Obama, is)]
[(Who, is), (Elizabeth, is)]
[(is, dbpedia), (down, dbpedia)]
[(WHO, IS), (EINSTEIN, IS)]
[(who, is), (einstein, is)]
[(Where, is), (capital, is)]
[(Where, is), (France, is)]
[(Who, is), (Allen, is)]
[(What, is), (Tetris, is)]
[(of, World)]
[(What, is), (World, is)]
[(Who, is), (Obama, is)]
[(what, is), (architecture, is)]
[(me, tell), (about, tell)]
[(me, tell), (about, tell)]
[(dbpedia, bot)]
[(you, suck)]
[(i, hate), (you, hate)]
[(fuc

[(old, is), (clinton, is)]
[(clause, is)]
[(what, are), (headlines, are)]
[(what, is), (price, is)]
[(what, is), (hi, is), (for, is)]
[(what, is), (news, is)]
[(what, 's), (speed, 's)]
[(what, 's), (price, 's)]
[(when, fly), (will, fly), (pigs, fly), (                         , fly)]
[(who, is), (president, is)]
[(who, was), (president, was)]
[(far, is), (it, is), (to, is)]
[(long, cats), (do, cats), (live, cats)]
[(how, long), (until, long)]
[(much, bitcoin), (is, bitcoin), (                                         , bitcoin)]
[(old, is), (stanley, is)]
[(old, is), (world, is)]
[(does, sound), (cat, sound), (like, sound)]
[(sos, stand), (for, stand)]
[(what, is), (car, is)]
[(what, is), (price, is)]
[(what, is), (mountain, is)]
[(what, is), (there, is), (eat, is)]
[(what, is), (score, is)]
[(what, 's), (price, 's)]
[(restaurants, are), (open, are)]
[(what, sounds), (good, sounds), (eat, sounds)]
[(what, 's), (score, 's)]
[(time, are), (games, are), (today, are)]
[(when, did), (war, di

...

[]
[(Albert_Einstein, entity)]
[(about, dbpedia)]
[]
[]
[]
[]
[(Who, is), (obama, is)]
[(Where, is), (Châteauroux, is)]
[(Where, is), (Châteauroux, is)]
[(Where, is), (Paris, is)]
[(Where, is), (Châteauroux, is)]
[]
[]
[(Planet, entity)]
[(more__http://dbpedia.org, learn), (resource, learn)]
[(similar__http://dbpedia.org, load), (resource, load)]
[(related__http://dbpedia.org, load), (resource, load)]
[(Who, is), (POTUS, is)]
[(Who, is), (president, is)]
[(Who, died), (have, died), (among, died)]
[(actors, died), (have, died), (among, died)]
[(far, is), (from, is)]
[(Putin, Is), (older, Is)]
[(Putin, Is), (older, Is), (Trump, Is)]
[(Moscow, Is), (older, Is)]
[(What, is), (difference, is)]
[(type, is), (Python, is)]
[(What, is), (classification, is)]
[(Which, exist), (languages, exist)]
[(languages, exist), (do, exist)]
[(languages, exist), (do, exist)]
[(languages, exist)]
[(What, exist), (languages, exist)]
[(What, are), (symptoms, are)]
[(What, is), (worth, is)]
[(What, are), (places

[(who, is), (serena, is), (Williams, is)]
[(where, born), (was, born), (williams, born)]
[(when, was), (williams, was)]
[(кто, шекспир)]
[(about, dbpedia)]
[(do, know), (you, know), (citites, know)]
[(what, do), (can, do), (you, do)]
[(Albert_Einstein, entity)]
[(of, list)]
[(me, show), (cities, show)]
[(more__http://dbpedia.org, learn), (Cities_of_the_world__Cities, learn)]
[(what, is), (it, is)]
[(about, dbpedia)]
[]
[]
[]
[(what, is), (google, is)]
[(what, is), (dbpedia, is)]
[(where, is), (capital, is)]
[(brazil, capital)]
[(Germany, capital)]
[(first, claimb), (to, claimb), (mont, claimb), (everest, claimb)]
[(first, claim), (to, claim), (mont, claim), (everest, claim)]
[(monte, evereste)]
[(monite, everiste)]
[(wife, barack)]
[(more__http://dbpedia.org, learn), (resource, learn)]
[(Michelle_Obama__Michelle, Obama)]
[(similar__http://dbpedia.org, load), (resource, load)]
[]
[(related__http://dbpedia.org, load), (resource, load)]
[]
[(obama, wife)]
[(where, is), (Paris, is)]
[]
[(w

[(Who, are), (creators, are)]
[(Who, is), (father, is)]
[(you, are), (from, are)]
[(You, are), (stupid, are)]
[(about, dbpedia)]
[(who, is), (person, is)]
[(what, is), (esperance, is)]
[(old, are), (Trump, are)]
[(what, is), (city, is)]
[(what, is), (city, is)]
[(what, is), (area, is)]
[(what, is), (name, is)]
[(what, is), (date, is), (today, is)]
[(when, start), (cup, start)]
[(when, start), (cup, start), (year, start)]
[(when, start), (cup, start)]
[(wher, is), (headquarter, is)]
[(university, based), (are, based), (in, based)]
[(employees, work), (with, work)]
[(what, is), (rank, is)]
[(who, is), (Obama, is)]
[(what, is), (name, is)]
[(person, who)]
[(who, is), (person, is)]
[(who, creat), (wikipedia, creat)]
[(india, kotla)]
[]
[(I, like), ('d, like), (know, like)]
[(old, is), (Steward, is)]
[]
[(old, are), (you, are)]
[(What, 's), (name, 's)]
[(What, is), (GDP, is)]
[(me, Tell), (something, Tell)]
[(Yes, do), (I, do)]
[(I, am)]
[]
[(song, Sing), (please, Sing)]
[(What, is), (song,

[(ho, is), (leader, is)]
[(Birth, place), (of, place)]
[(What, is), (Buddhism, is)]
[(Who, is), (father, is)]
[(DBPedia, Hi)]
[]
[(How, are), (you, are)]
[]
[(old, are), (you, are)]
[(La, padrone), (voce, padrone), (del, padrone)]
[(Giuseppe, Ribezz)]
[]
[(load, Joseph","count":68,"limit":5,"offset":5), (resource, Joseph","count":68,"limit":5,"offset":5)]
[(carafa, lecce)]
[]
[(time, is), (it, is)]
[(get, location__Berlin)]
[(Where, is), (Popayan, is)]
[(Is, endpoint), (DBpedia, endpoint), (up, endpoint)]
[(people, live), (in, live)]
[(Who, is), (president, is)]
[(what, is), (sefirah, is)]
[(what, is), (kabbala, is)]
[(where, is), (berlin, is)]
[(was, rabbi), (akiva, rabbi)]
[(who, is), (obama, is)]
[]
[(who, is), (trump, is)]
[(who, is), (einstein, is)]
[(who, is), (einstein, is)]
[(what, is), (planet, is)]
[(about, dbpedia)]
[(what, is), (dbpedia, is)]
[(who, is), (renzi, is)]
[(where, is), (rome, is)]
[]
[(What, is), (Visaginas, is)]
[(Where, is), (Lithuania, is)]
[(Albert_Einstein,

KeyboardInterrupt: 

In [ ]:
# TODO:
# implement clustering 
# request -> apply NER to check the entity-type (better person recognizers or better location recognizers)

#### Adding code for NER via spacy and spotlight to detect location vs person.

In [13]:
def hit_spotlight(text:str,url:str = 'https://api.dbpedia-spotlight.org/en/annotate'):
    params = {'text': text}
    req = requests.get(url, params=params) # May be add some logic for fall back
    return req.content

def get_spotlight_res(sent: str):
    soup = BeautifulSoup(hit_spotlight(text=sent).decode('utf-8'),'html.parser')
    a_tags = soup.find_all('a')
    
    # Just to find the types of each val
    a_types = [[val.get('typeof'),val.get('about')] for val in a_tags]
    
    # return a_types
    return a_types

def check_type_spotlight(sent:str, dbp):
    
    a_types = get_spotlight_res(sent)    
    is_person = False
    is_place = False
    
    for a in a_types:
        class_path = dbp.get_type_of_resource(a[1])
        class_path = [c.decode('utf-8') for c in class_path]
        
        # Can be made efficient by combining them somehow. 
        
        for c in class_path:
            if PERSON in c:
                is_person = True
                break
        
        for c in class_path:
            if LOCATION in c:
                is_place = True
                break
        
    if is_person and not is_place:
        return "person"
    
    if is_place and not is_person:
        return "location"
    
    if is_place and is_person:
        return "both"
    
    return False

def get_spacy_res(sent):
    doc = nlp(sent)
    for tok in doc:
        print(tok, tok.ent_type, tok.ent_type_)

def check_type_ner(sent:str):
    doc = nlp(sent)
    ent_type = [tok.ent_type_ for tok in doc]
    number_of_person_ent = ent_type.count('PERSON')
    number_of_location_ent = ent_type.count('GPE')
    if number_of_location_ent > number_of_person_ent:
        return "location"
    elif number_of_person_ent > number_of_location_ent:
        return "person"
    return False

In [18]:

# Check if the cache exists and if not create a new one
cache = False
if os.path.exists('data/type_meta_data.json'):
    print("cache exists")
    new_final_meta_data = json.load(open('data/type_meta_data.json'))
    cache = True
    
    # Simple check to see if the length of unanswered_ques is same as type_data. if Not create a new one
    if len(type_data) != len(unanswered_ques):
        print(f"length of unanswered question ({len(unanswered_ques)}) does not match with cache data {len(type_data)}")
        cache = False

if cache == 'False':
    # create new cache
    
    final_meta_data = []
    for ques in tqdm(unanswered_ques):
        try:
            final_meta_data.append([check_type_spotlight(sent=ques, dbp=dbp),
                               check_type_ner(sent=ques)])
        except:
            final_meta_data.append([])
    
    # Loop is redundant. Can be combined with the loop above
    new_final_meta_data = []
    for i,j in zip(final_meta_data,unanswered_ques):
        try:
            temp = {
                'spotlight': i[0],
                'spacy_ner': i[1],
                'statment': j
            }
        except:
            temp = {
                'spotlight': 'NAN',
                'spacy_ner': 'NAN',
                'statment': j
            }
        new_final_meta_data.append(temp)
    
    # store the cache
    json.dump(new_final_meta_data,open('type_meta_data.json', 'w+'))

cache exists


In [39]:
def count_type(key):
    # Snippet for analysis
    p_c = 0 # person count
    l_c = 0 # location count
    both = 0 # valid for dbpedia. 
    #DBpedia entity type is both a person as well as location. For example United States of America 
    none_of_them = 0

    for f in new_final_meta_data:
        try:
            if f[key] == 'both':
                both = both + 1
            if f[key] == 'person':
                p_c = p_c + 1
            elif f[key] == 'location':
                l_c = l_c + 1
            else :
                none_of_them = none_of_them + 1
        except:
            exception_counter = exception_counter + 1
            continue
    

    return p_c,l_c,none_of_them,both

key = 'spotlight' # choose between spotlight; spacy_ner
p_c,l_c,none_of_them,both = count_type(key)

print(f"method used {key}")
print(f"number of statments with entity type detected as person {p_c}")
print(f"number of statments with entity type detected as location {l_c}")
print(f"number of statments with entity type detected as both {both}")
print(f"number of statments with entity type detected as none of them {none_of_them}")

print("**************")
print()

key = 'spacy_ner' # choose between spotlight; spacy_ner
p_c,l_c,none_of_them,both = count_type(key)

print(f"method used {key}")
print(f"number of statments with entity type detected as person {p_c}")
print(f"number of statments with entity type detected as location {l_c}")
print(f"number of statments with entity type detected as both {both}")
print(f"number of statments with entity type detected as none of them {none_of_them}")

method used spotlight
number of statments with entity type detected as person 650
number of statments with entity type detected as location 458
number of statments with entity type detected as both 76
number of statments with entity type detected as none of them 2878
**************

method used spacy_ner
number of statments with entity type detected as person 483
number of statments with entity type detected as location 345
number of statments with entity type detected as both 0
number of statments with entity type detected as none of them 3158
